# HTTP Server in Python with Flask

You can download the material at [https://tinyurl.com/session5-distsys](https://tinyurl.com/session5-distsys)

## The simplest Flask example:

[Flask](https://flask.palletsprojects.com/en/stable/) is a lightweight web application framework that allows to quickly setup a web server or web applications in Python

In [3]:
from flask import Flask

app = Flask(__name__)

# Route decorator used to select the resource specified with the URL
@app.route("/")
def welcome():
    return "<h1>The server is up and running!</h1>\n"

@app.route("/test/")
def test():
    return "<h1>This is the \"test\" folder.</h1>\n"


if __name__ == '__main__':
 
    app.run(host="0.0.0.0", port=5001, debug=False, threaded=True)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://10.200.7.216:5001
Press CTRL+C to quit
127.0.0.1 - - [24/Oct/2025 10:46:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2025 10:47:31] "GET / HTTP/1.1" 200 -


## How can we test the server from the command line?

cURL is a command line tool and library for transferring data with URLs to and from servers. It supports many application-layer protocols, including HTTP.

Here's a quick rundown of what curl can do:

- **Download files**: This is probably the most common use case. You can simply point curl to a URL, and it will download the file to your local machine.
- **Upload files**: Not just for downloads, curl can also upload files to servers using protocols like FTP.
- **Interact with web servers**: curl can be used to send HTTP requests (GET, POST, etc.) to web servers and retrieve data in various formats like HTML, JSON, or XML.

In this lab session we will use it to test the beahiour of our flask server (for Windows pc it is suggested to launch it via WSL). 

For a simple GET request:
`curl http://127.0.0.1:5001`

To print also the HTTP response header, add the `-i` option:
`curl -i http://127.0.0.1:5001`

## How to define a REST API endpoint

In [4]:
from flask import Flask, request

app = Flask(__name__)

users = {}
users['users'] = []

# Route decorator to specify an endpoint and 
# what to do when a POST method is received
@app.route('/people/v1/users', methods=['POST'])
def create_user():
  # Check if request has content type as JSON
  if request.is_json:

    # Get the JSON data from the request
    user_data = request.get_json()

    # Access data from the JSON
    name = user_data.get('name')
    email = user_data.get('email')
    if not name or not email:
      return "Missing required fields (name and email)\n", 400  # Bad request

    users['users'].append({'name': name, 'email': email})

    # Return a success response
    return f"User created successfully! Name: {name}, Email: {email}\n", 201
  
  else:
    # Return error if not JSON
    return "Request must be JSON\n", 415
  
# Route decorator used to select the resource specified with the URL
@app.route("/")
def welcome():
    return "<h1>The server is up and running!</h1>\n"

@app.route("/test/")
def test():
    return "<h1>This is the \"test\" folder.</h1>\n"

if __name__ == '__main__':
  app.run(host="0.0.0.0", port=5001, debug=False, threaded=True)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://10.200.7.216:5001
Press CTRL+C to quit
127.0.0.1 - - [24/Oct/2025 11:01:40] "POST /people/v1/users HTTP/1.1" 201 -
127.0.0.1 - - [24/Oct/2025 11:02:33] "POST /people/v1/users HTTP/1.1" 400 -
127.0.0.1 - - [24/Oct/2025 11:03:28] "POST /people/v1/users HTTP/1.1" 415 -


Again for testing we can use curl.
To POST a JSON file to a server the syntax is like this:

`curl -i -X POST -H "Content-Type: application/json" -d '{"name": "John", "email": "john@example.com"}' http://127.0.0.1:5001/people/v1/users`


But how do we see the list of registered users?

In [5]:
from flask import Flask, request, jsonify

app = Flask(__name__)

users = {}
users['users'] = []

# Route decorator to specify an endpoint and 
# what to do when a POST or GET method is received
@app.route('/people/v1/users', methods=['GET', 'POST'])
def user_handler():

  if request.method == 'POST':
    # Check if request has content type as JSON
    if request.is_json:

      # Get the JSON data from the request
      user_data = request.get_json()

      # Access data from the JSON
      name = user_data.get('name')
      email = user_data.get('email')
      if not name or not email:
        return "Missing required fields (name and email)\n", 400  # Bad request

      users['users'].append({'name': name, 'email': email})

      # Return a success response
      return f"User created successfully! Name: {name}, Email: {email}\n", 201
  
    else:
      # Return error if not JSON
      return "Request must be JSON\n", 415
    
  else:
    # Handle GET request - return list of users
    return jsonify(users), 200  # Return users data in JSON format
  
# Route decorator used to select the resource specified with the URL
@app.route("/")
def welcome():
    return "<h1>The server is up and running!</h1>\n"

@app.route("/test/")
def test():
    return "<h1>This is the \"test\" folder.</h1>\n"

if __name__ == '__main__':
  app.run(host="0.0.0.0", port=5001, debug=False, threaded=True)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://10.200.7.216:5001
Press CTRL+C to quit
127.0.0.1 - - [24/Oct/2025 11:06:53] "POST /people/v1/users HTTP/1.1" 201 -
127.0.0.1 - - [24/Oct/2025 11:07:31] "GET /people/v1/users HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2025 11:07:54] "POST /people/v1/users HTTP/1.1" 201 -
127.0.0.1 - - [24/Oct/2025 11:08:00] "GET /people/v1/users HTTP/1.1" 200 -


Now we can retrieve the list of users with a simple GET to the endpoint:

`curl -i http://127.0.0.1:5001/people/v1/users`

But what if we want to see the details of a single user?

We have two ways: RESTful and non-RESTful

## The non-RESTful way (i.e., HTTP options)

The non-RESTful method employs URL parameters to pass as an argument the name of the user we want to GET:

`curl -i http://127.0.0.1:5001/people/v1/users?name=John`

However, this method does not allow the server to uniquely identify a given resource, as the value of the `name` field could be the same for multiple resources.

In [6]:
from flask import Flask, request, jsonify

app = Flask(__name__)

users = {}
users['users'] = []

# Route decorator to specify an endpoint and 
# what to do when a POST or GET method is received
@app.route('/people/v1/users', methods=['GET', 'POST'])
def user_handler():

  if request.method == 'POST':
    # Check if request has content type as JSON
    if request.is_json:

      # Get the JSON data from the request
      user_data = request.get_json()

      # Access data from the JSON
      name = user_data.get('name')
      email = user_data.get('email')
      if not name or not email:
        return "Missing required fields (name and email)\n", 400  # Bad request

      users['users'].append({'name': name, 'email': email})

      # Return a success response
      return f"User created successfully! Name: {name}, Email: {email}\n", 201
  
    else:
      # Return error if not JSON
      return "Request must be JSON\n", 415
    
  else:
    # Handle GET request - return list of users
    name = request.args.get('name')  # Get name from query parameter

    if name:
      # Filter users based on name
      filtered_users = [user for user in users['users'] if user['name'] == name]

      if filtered_users:
        return jsonify(filtered_users), 200
      
      else:
        return f"User with name '{name}' not found\n", 404
      
    else:
      return jsonify(users), 200  # Return users data in JSON format

  
# Route decorator used to select the resource specified with the URL
@app.route("/")
def welcome():
    return "<h1>The server is up and running!</h1>\n"

@app.route("/test/")
def test():
    return "<h1>This is the \"test\" folder.</h1>\n"

if __name__ == '__main__':
  app.run(host="0.0.0.0", port=5001, debug=False, threaded=True)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://10.200.7.216:5001
Press CTRL+C to quit
127.0.0.1 - - [24/Oct/2025 11:14:41] "GET /people/v1/users?name=John HTTP/1.1" 404 -
127.0.0.1 - - [24/Oct/2025 11:15:11] "POST /people/v1/users HTTP/1.1" 201 -
127.0.0.1 - - [24/Oct/2025 11:15:18] "GET /people/v1/users?name=John HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2025 11:15:41] "POST /people/v1/users HTTP/1.1" 201 -
127.0.0.1 - - [24/Oct/2025 11:15:51] "POST /people/v1/users HTTP/1.1" 201 -
127.0.0.1 - - [24/Oct/2025 11:15:56] "GET /people/v1/users?name=John HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2025 11:16:19] "GET /people/v1/users HTTP/1.1" 200 -


## The RESTful way

### UUIDs: Universally Unique IDentifiers

Universally Unique Identifiers (UUIDs) are 128-bit values represented as hexadecimal strings. They play a crucial role in ensuring the uniqueness of data across systems and applications.

###  Understanding the Versions:

There are five defined versions of UUIDs, each with a distinct generation method:

1. **UUID1 (Time-based):** Uses the current time, MAC address, and a sequence number. While commonly used, it can potentially leak information about the system generating the UUID.
2. **UUID3 (Name-based - MD5):** Generates a UUID based on a namespace identifier (like a domain name) and a name, hashed with the MD5 algorithm. Potential collisions exist if different names map to the same hash.
3. **UUID4 (Random):** Employs a cryptographically strong pseudo-random number generator, offering the highest level of uniqueness and no reliance on external information. This is the most widely used version.
4. **UUID5 (Name-based - SHA-1):** Similar to UUID3, but uses the SHA-1 hashing algorithm.
5. **Namespace variant (reserved for future use):** Reserved for potential future versions based on namespaces.

###  How UUID4 Works:

UUID4 relies on a combination of:

* **Randomness:** A cryptographically secure pseudo-random number generator (CSPRNG) is the core for generating unpredictable values.
* **Clock sequence:** A portion of the UUID reflects the system's time to avoid duplication within short time intervals.
* **Node identifier:**  This can be derived from the network interface's MAC address, but version 4 specification allows for modification to enhance privacy.

These elements are combined using bitwise operations to create a unique 128-bit value. The final string representation is a hexadecimal string with specific formatting (e.g., `xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx`).

###  Benefits of UUID4:

* **Guaranteed Uniqueness:** The probability of collision with another randomly generated UUID4 is extremely low, making them ideal for scenarios where unique identification is crucial.
* **Decentralization:** No central authority is required for generation. Any system can create UUID4s independently.
* **Simplicity:** Human-readable string format simplifies debugging and logging compared to raw binary identifiers.


###  Drawbacks of UUID4:

* **Storage Space:** Compared to simpler identifiers (e.g., integers), UUIDs require more storage space due to their string representation.
* **Performance:** Generating and comparing UUIDs can be computationally more expensive than simpler alternatives.

Overall, UUID4 offers a strong balance between uniqueness, decentralization, and readability.  However, for situations where storage space or performance is a critical concern, alternative identifier schemes might be more suitable.



With the RESTful method we can specify in the endpoint the unique identifier of any existing resource:

`curl -i http://127.0.0.1:5001/people/v1/users/xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx`


In [7]:
import uuid
from flask import Flask, request, jsonify

app = Flask(__name__)

users = {}
users['users'] = []

# Function to generate a unique UUID4
def generate_uuid():
  return str(uuid.uuid4())

# Define endpoint and method to get all users (GET on /users)
@app.route('/people/v1/users', methods=['GET'])
def get_all_users():
  return jsonify(users), 200  # Return list of users as JSON

# Define endpoint and method to create a new user (POST on /users)
@app.route('/people/v1/users', methods=['POST'])
def create_user():

  # Check if request has content type as JSON
  if request.is_json:
      
    # Get the JSON data from the request
    user_data = request.get_json()
    
    # Access data from the JSON
    name = user_data.get('name')
    email = user_data.get('email')
    if not name or not email:
      return "Missing required fields (name and email)\n", 400  # Bad request
    
    # Generate a unique UUID for the user
    user_id = generate_uuid()

    users['users'].append({'id': user_id, 'name': name, 'email': email})

    # Return a success response
    return f"User created successfully! ID: {user_id}, Name: {name}, Email: {email}\n", 201

  else:
    # Return error if not JSON
    return "Request must be JSON\n", 415
 
# Define endpoint and method to get a specific user by ID (GET on /users/<user_id>)
@app.route('/people/v1/users/<string:user_id>', methods=['GET'])
def get_user_by_id(user_id):

  for user in users['users']:
    if user.get('id') == user_id:
      return jsonify(user), 200
  return f"User with ID {user_id} not found\n", 404  # Not Found

# Define endpoint and method to delete a specific user (DELETE on /users/<user_id>)
@app.route('/people/v1/users/<string:user_id>', methods=['DELETE'])
def delete_user_by_id(user_id):

  for i, user in enumerate(users['users']):
    if user.get('id') == user_id:
      del users['users'][i]  # Remove user from the list
      return f"User with ID {user_id} deleted successfully\n", 204  # No Content
  return f"User with ID {user_id} not found\n", 404  # Not Found
  
# Route decorator used to select the resource specified with the URL
@app.route("/")
def welcome():
    return "<h1>The server is up and running!</h1>\n"

@app.route("/test/")
def test():
    return "<h1>This is the \"test\" folder.</h1>\n"

if __name__ == '__main__':
  app.run(host="0.0.0.0", port=5001, debug=False, threaded=True)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://10.200.7.216:5001
Press CTRL+C to quit
127.0.0.1 - - [24/Oct/2025 11:31:27] "POST /people/v1/users HTTP/1.1" 201 -
127.0.0.1 - - [24/Oct/2025 11:32:25] "GET /people/v1/users/864d82a5-e4e2-4b17-8358-d0d66d34e470 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2025 11:32:52] "DELETE /people/v1/users/864d82a5-e4e2-4b17-8358-d0d66d34e470 HTTP/1.1" 204 -
127.0.0.1 - - [24/Oct/2025 11:33:06] "DELETE /people/v1/users/864d82a5-e4e2-4b17-8358-d0d66d34e470 HTTP/1.1" 404 -
127.0.0.1 - - [24/Oct/2025 11:33:09] "GET /people/v1/users/864d82a5-e4e2-4b17-8358-d0d66d34e470 HTTP/1.1" 404 -


## Extra: OpenAPI specifications

The [OpenAPI Specification](https://www.openapis.org/) is a widely adopted industry standard for describing APIs (Application Programming Interfaces). It provides a machine-readable format for documenting API functionality, making it easier for developers to understand and interact with your services.

Here's what OpenAPI offers:

* **Standardized Documentation:**  
  OpenAPI defines a common language for describing APIs, regardless of the underlying technology. This allows developers to quickly grasp the API's capabilities without needing to decipher custom documentation formats.
* **Improved Developer Experience:** 
  By leveraging OpenAPI, you can generate interactive API documentation that includes code samples, interactive testing tools, and clear explanations of endpoints, parameters, and responses. This empowers developers to integrate with your API more efficiently.
* **Increased Collaboration:** 
  Standardized API descriptions foster better communication between API providers and consumers. Developers can clearly see what data the API expects and what data it returns, streamlining integration and reducing errors.
* **Automated Tools:** 
  OpenAPI specifications can be used to generate code clients (libraries) in various programming languages. This saves developers time and effort by automating the process of interacting with your API.


Overall, OpenAPI contributes to a more efficient and productive API ecosystem. It empowers developers to discover, understand, and integrate with your APIs more seamlessly, fostering collaboration and innovation.


For example, 3GPP has standardized all the API exposed by services of the 5G core network in OpenAPI format. For example, [to manage PDU sessions](https://jdegre.github.io/editor/?url=https://raw.githubusercontent.com/jdegre/5GC_APIs/master/TS29502_Nsmf_PDUSession.yaml).
